# Calculations for optimized algorithm for linear feed and exponential/linear feed

In this notebook, symbolic calculation are performed to solve the differential equations without a numerical solver. The production rate is dependent on the available glucose.

## Linear feed

In [1]:
import sympy as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
qP, qX, qm, Gf, qf, qp_max, km, mu, Yxg, Ypg, qG, X, X0, t= sp.symbols('q_P q_X q_m G_f q_f q_p^{max} k_m mu Y_{X/G} Y_{P/G} q_G X X_0 t', positiv=True)
X = sp.Function('X', positiv=True)(t)
c1, c2, c3, c4, c5 = sp.symbols('c_1 c_2 c_3 c_4 c_5')

### Step 1: Find a definition of $q_P$ only dependent on the growth rate and model parameter

In [3]:
qP_def = sp.Eq(qP, qp_max / (km / (qG - qm) + 1))
qP_def

Eq(q_P, q_p^{max}/(k_m/(q_G - q_m) + 1))

In [4]:
qG_def = sp.Eq(qG, mu / Yxg + qP / Ypg + qm)
qG_def

Eq(q_G, q_m + mu/Y_{X/G} + q_P/Y_{P/G})

In [5]:
qP_eq = sp.Eq(qP, qp_max / (km / (mu / Yxg + qP / Ypg) + 1))
qP_eq

Eq(q_P, q_p^{max}/(k_m/(mu/Y_{X/G} + q_P/Y_{P/G}) + 1))

In [6]:
qP_eq2 = sp.Eq(qP, sp.solve(qP_eq, qP)[1]).simplify()
qP_eq2

Eq(q_P, (-Y_{P/G}*Y_{X/G}*k_m - Y_{P/G}*mu + Y_{X/G}*q_p^{max} + sqrt(Y_{P/G}**2*Y_{X/G}**2*k_m**2 + 2*Y_{P/G}**2*Y_{X/G}*k_m*mu + Y_{P/G}**2*mu**2 - 2*Y_{P/G}*Y_{X/G}**2*k_m*q_p^{max} + 2*Y_{P/G}*Y_{X/G}*mu*q_p^{max} + Y_{X/G}**2*q_p^{max}**2))/(2*Y_{X/G}))

In [7]:
qP_eq2.rhs.collect(mu)

(-Y_{P/G}*Y_{X/G}*k_m - Y_{P/G}*mu + Y_{X/G}*q_p^{max} + sqrt(Y_{P/G}**2*Y_{X/G}**2*k_m**2 + Y_{P/G}**2*mu**2 - 2*Y_{P/G}*Y_{X/G}**2*k_m*q_p^{max} + Y_{X/G}**2*q_p^{max}**2 + mu*(2*Y_{P/G}**2*Y_{X/G}*k_m + 2*Y_{P/G}*Y_{X/G}*q_p^{max})))/(2*Y_{X/G})

In [8]:
qG_eq = qG_def.subs(qP, qP_eq2.rhs).simplify()
qG_eq

Eq(q_G, -k_m/2 + q_m + mu/(2*Y_{X/G}) + q_p^{max}/(2*Y_{P/G}) + sqrt(Y_{P/G}**2*Y_{X/G}**2*k_m**2 + 2*Y_{P/G}**2*Y_{X/G}*k_m*mu + Y_{P/G}**2*mu**2 - 2*Y_{P/G}*Y_{X/G}**2*k_m*q_p^{max} + 2*Y_{P/G}*Y_{X/G}*mu*q_p^{max} + Y_{X/G}**2*q_p^{max}**2)/(2*Y_{P/G}*Y_{X/G}))

In [9]:
sp.Eq(qG, qf * Gf / X)

Eq(q_G, G_f*q_f/X(t))

In [10]:
mu_def = qG_eq.subs(qG, qf * Gf / X).simplify()
mu_def

Eq(G_f*q_f/X(t), (Y_{P/G}*Y_{X/G}*(-k_m + 2*q_m) + Y_{P/G}*mu + Y_{X/G}*q_p^{max} + sqrt(Y_{P/G}**2*Y_{X/G}**2*k_m**2 + 2*Y_{P/G}**2*Y_{X/G}*k_m*mu + Y_{P/G}**2*mu**2 - 2*Y_{P/G}*Y_{X/G}**2*k_m*q_p^{max} + 2*Y_{P/G}*Y_{X/G}*mu*q_p^{max} + Y_{X/G}**2*q_p^{max}**2))/(2*Y_{P/G}*Y_{X/G}))

In [11]:
mu_eq = sp.Eq(mu, sp.solve(mu_def, mu)[0].simplify())
mu_eq.rhs.collect(X)

Y_{X/G}*(G_f**2*Y_{P/G}*q_f**2 + (-Y_{P/G}*k_m*q_m + Y_{P/G}*q_m**2 + q_m*q_p^{max})*X(t)**2 + (G_f*Y_{P/G}*k_m*q_f - 2*G_f*Y_{P/G}*q_f*q_m - G_f*q_f*q_p^{max})*X(t))/(Y_{P/G}*(G_f*q_f + (k_m - q_m)*X(t))*X(t))

### Step 2: Solve equation for $X$

With the formula for $mu$ the differential equation for the biomass $\frac{dX}{dt} = \mu X$ can be solved.

In [12]:
ode_X = sp.Eq(sp.Derivative(X, t), mu_eq.rhs * X)
ode_X

Eq(Derivative(X(t), t), Y_{X/G}*(G_f**2*Y_{P/G}*q_f**2 + G_f*Y_{P/G}*k_m*q_f*X(t) - 2*G_f*Y_{P/G}*q_f*q_m*X(t) - G_f*q_f*q_p^{max}*X(t) - Y_{P/G}*k_m*q_m*X(t)**2 + Y_{P/G}*q_m**2*X(t)**2 + q_m*q_p^{max}*X(t)**2)/(Y_{P/G}*(G_f*q_f + k_m*X(t) - q_m*X(t))))

In [13]:
def_dX = c1 * (X**2 + c2 * X + c3) / (c4 + X)
def_dX

c_1*(c_2*X(t) + c_3 + X(t)**2)/(c_4 + X(t))

In [14]:
sp.diff(def_dX, X).simplify()

c_1*(-c_2*X(t) - c_3 + (c_2 + 2*X(t))*(c_4 + X(t)) - X(t)**2)/(c_4 + X(t))**2

In [15]:
sp.diff(sp.diff(def_dX, X),X).simplify()

2*c_1*(-c_2*c_4 + c_3 + c_4**2)/(c_4**3 + 3*c_4**2*X(t) + 3*c_4*X(t)**2 + X(t)**3)

In [16]:
sp.integrate(1 / ode_X.rhs, X).simplify()

Y_{P/G}*(-k_m*(-Y_{P/G}*k_m + Y_{P/G}*q_m + q_p^{max})*log((-G_f*q_f + q_m*X(t))/q_m) + q_m*q_p^{max}*log((G_f*Y_{P/G}*q_f + Y_{P/G}*k_m*X(t) - Y_{P/G}*q_m*X(t) - q_p^{max}*X(t))/(Y_{P/G}*k_m - Y_{P/G}*q_m - q_p^{max})))/(Y_{X/G}*q_m*(Y_{P/G}*k_m - q_p^{max})*(-Y_{P/G}*k_m + Y_{P/G}*q_m + q_p^{max}))

In [17]:
ode_sol = sp.dsolve(ode_X)
ode_sol

Eq(-k_m*log(G_f*q_f*(-Y_{P/G}**2*k_m**3/(q_m*(Y_{P/G}*k_m - q_p^{max})) + 2*Y_{P/G}*k_m**2*q_p^{max}/(q_m*(Y_{P/G}*k_m - q_p^{max})) + Y_{P/G}*k_m - k_m*q_p^{max}**2/(q_m*(Y_{P/G}*k_m - q_p^{max})) + q_p^{max})/(Y_{P/G}*k_m**2 - Y_{P/G}*k_m*q_m - k_m*q_p^{max} - q_m*q_p^{max}) + X(t))/(q_m*(Y_{P/G}*k_m - q_p^{max})) - q_p^{max}*log(G_f*q_f*(-Y_{P/G}**2*k_m**2*q_p^{max}/((Y_{P/G}*k_m - q_p^{max})*(Y_{P/G}*k_m - Y_{P/G}*q_m - q_p^{max})) + 2*Y_{P/G}*k_m*q_p^{max}**2/((Y_{P/G}*k_m - q_p^{max})*(Y_{P/G}*k_m - Y_{P/G}*q_m - q_p^{max})) + Y_{P/G}*k_m - q_p^{max}**3/((Y_{P/G}*k_m - q_p^{max})*(Y_{P/G}*k_m - Y_{P/G}*q_m - q_p^{max})) + q_p^{max})/(Y_{P/G}*k_m**2 - Y_{P/G}*k_m*q_m - k_m*q_p^{max} - q_m*q_p^{max}) + X(t))/((Y_{P/G}*k_m - q_p^{max})*(Y_{P/G}*k_m - Y_{P/G}*q_m - q_p^{max})) - Y_{X/G}*t/Y_{P/G}, C1)

While this equation is not solvable, we get an expression of X that can be solved by a root finding algorithm. As most of the variables are constant, the equation can be simplified. The definition of the constants is described in the thesis.

The expression in the logarithms are created by integration $\int \frac{1}{x} = \log(x)$. Therefore, we have to multiply the expression in the logarithms by $-1$, if they are negative.

In [18]:
a, b, c, d, e, f, g = sp.symbols('a b c d e f g')
sol_simplified = sp.Eq (a * sp.log(b + X) - c * sp.log(d + X) - e * t, f)
sol_simplified

Eq(a*log(b + X(t)) - c*log(d + X(t)) - e*t, f)

## Exponential linear feed

In this case the growth rate is given, this makes the equation solvable.

For the starvation phase we still use a theoretical growth rate, as the production rate is equal to the production rate in the growth phase with the same glucose uptake.

Now we just have to use the equation for $q_P$ to solve the equation. The derivation of the solutions for X, P and V are shown in the thesis.

In [19]:
qP_eq2

Eq(q_P, (-Y_{P/G}*Y_{X/G}*k_m - Y_{P/G}*mu + Y_{X/G}*q_p^{max} + sqrt(Y_{P/G}**2*Y_{X/G}**2*k_m**2 + 2*Y_{P/G}**2*Y_{X/G}*k_m*mu + Y_{P/G}**2*mu**2 - 2*Y_{P/G}*Y_{X/G}**2*k_m*q_p^{max} + 2*Y_{P/G}*Y_{X/G}*mu*q_p^{max} + Y_{X/G}**2*q_p^{max}**2))/(2*Y_{X/G}))

In [20]:
qP_eq2.rhs.collect(mu)

(-Y_{P/G}*Y_{X/G}*k_m - Y_{P/G}*mu + Y_{X/G}*q_p^{max} + sqrt(Y_{P/G}**2*Y_{X/G}**2*k_m**2 + Y_{P/G}**2*mu**2 - 2*Y_{P/G}*Y_{X/G}**2*k_m*q_p^{max} + Y_{X/G}**2*q_p^{max}**2 + mu*(2*Y_{P/G}**2*Y_{X/G}*k_m + 2*Y_{P/G}*Y_{X/G}*q_p^{max})))/(2*Y_{X/G})

In [21]:
(qP_eq2.rhs * 2* Yxg / Ypg).simplify().collect(mu)

-Y_{X/G}*k_m - mu + Y_{X/G}*q_p^{max}/Y_{P/G} + sqrt(Y_{P/G}**2*Y_{X/G}**2*k_m**2 + Y_{P/G}**2*mu**2 - 2*Y_{P/G}*Y_{X/G}**2*k_m*q_p^{max} + Y_{X/G}**2*q_p^{max}**2 + mu*(2*Y_{P/G}**2*Y_{X/G}*k_m + 2*Y_{P/G}*Y_{X/G}*q_p^{max}))/Y_{P/G}

## Calculate $\mu$ for $q_P^{\mu}$
Usually $\mu$ is not calculated in the model. Getting the growth rate from available variables requires solving a quadratic equation.

In [22]:
qP_min, qP_max, kM = sp.symbols('q_P^{\min} q_P^{\max} k_M')
qP_mu_def = sp.Eq(qP, qP_min + qP_max * mu / (kM + mu))
qP_mu_def

Eq(q_P, mu*q_P^{\max}/(k_M + mu) + q_P^{\min})

In [23]:
qG_mu = qG_def.subs(qP, qP_mu_def.rhs)
qG_mu

Eq(q_G, q_m + mu/Y_{X/G} + (mu*q_P^{\max}/(k_M + mu) + q_P^{\min})/Y_{P/G})

In [24]:
mu_def = sp.solve(qG_mu, mu)[1].simplify().collect(Ypg).collect(Yxg)
mu_def

(Y_{P/G}*(Y_{X/G}*(q_G - q_m) - k_M) + Y_{X/G}*(-q_P^{\max} - q_P^{\min}) + sqrt(Y_{P/G}**2*(Y_{X/G}**2*(q_G**2 - 2*q_G*q_m + q_m**2) + Y_{X/G}*(2*k_M*q_G - 2*k_M*q_m) + k_M**2) + Y_{P/G}*(Y_{X/G}**2*(-2*q_G*q_P^{\max} - 2*q_G*q_P^{\min} + 2*q_P^{\max}*q_m + 2*q_P^{\min}*q_m) + Y_{X/G}*(2*k_M*q_P^{\max} - 2*k_M*q_P^{\min})) + Y_{X/G}**2*(q_P^{\max}**2 + 2*q_P^{\max}*q_P^{\min} + q_P^{\min}**2)))/(2*Y_{P/G})

In [27]:
((qG_mu.rhs - qG_mu.lhs) * (kM + mu) * Yxg * Ypg).simplify().expand().collect(mu)

-Y_{P/G}*Y_{X/G}*k_M*q_G + Y_{P/G}*Y_{X/G}*k_M*q_m + Y_{P/G}*mu**2 + Y_{X/G}*k_M*q_P^{\min} + mu*(-Y_{P/G}*Y_{X/G}*q_G + Y_{P/G}*Y_{X/G}*q_m + Y_{P/G}*k_M + Y_{X/G}*q_P^{\max} + Y_{X/G}*q_P^{\min})